# Introduction of Linear decision trees

In this jupyter notebook, I am going to discuss how Linear decision trees can be added to Gogeta

In [450]:
# import necessary libraries 
using EvoTrees
using JuMP
using Gurobi

The key difference is that Linear decision trees have a linear function at the leafs, while normal decision tree has a coefficient. Lest create a dataset to learn a simple function:
$$f(x) = \sum_{i=1}^5 x_i^3$$



In [451]:
data = rand(1000, 5) .- 0.5;
x_train = data[1:750, :];
y_train = vec(sum(map.(x->x^3, x_train), dims=2));

x_test = data[751:end, :];
y_test = vec(sum(map.(x->x^3, x_test), dims=2));

Now using `EvoTrees`, lets create forest of 2 trees 

In [452]:
config = EvoTreeRegressor(nrounds=200, max_depth=5);
evo_model = fit_evotree(config; x_train, y_train, verbosity=0);

The first tree is always bias tree

In [453]:
evo_model.trees[1]

EvoTrees.Tree{EvoTrees.MSE, 1}
 - feat: [0]
 - cond_bin: UInt8[0x00]
 - cond_float: Any[0.0]
 - gain: [0.0]
 - pred: Float32[-0.0026170067;;]
 - split: Bool[0]


These are predictions learned by general tree. With Linear decision tree, we would also have $R^n$ vector for each leaf in the tree. For now lets assume that all gradients are zeros and coefficients equal to predictions of `EvoTrees` 

In [454]:
leaves = findall(node -> evo_model.trees[2].split[node] == false && (node == 1 || evo_model.trees[2].split[floor(Int, node / 2)] == true), 1:length(evo_model.trees[2].split))

evo_model.trees[2].pred[leaves]

15-element Vector{Float32}:
 -0.032087997
 -0.025356872
 -0.03195912
 -0.022056153
 -0.010642417
 -0.0074923313
  0.0033112094
 -0.021318046
 -0.0058930465
 -0.00015210488
  0.008136735
  0.0033859473
  0.011718282
  0.0068052234
  0.015342223

If Linear decision trees to be introduced in Julia, I would expect them to have similar structure as `EvoTrees`. 

In [455]:
struct LTEModel
    n_trees::Int64
    n_feats::Int64
    n_leaves::Array{Int64}
    leaves::Array{Array}
    splits::Matrix{Any}
    splits_ordered::Array{Vector}
    n_splits::Array{Int64}
    a::Array{Array}
    b::Array{Array}
    split_nodes::Array{Array}
    child_leaves::Array{Array}
end

In [456]:
function extract_evotrees_info(evo_model; tree_limit=length(evo_model.trees))

    n_trees = tree_limit
    n_feats = length(evo_model.info[:fnames])

    n_leaves = Array{Int64}(undef, n_trees) # number of leaves on each tree
    leaves = Array{Array}(undef, n_trees) # ids of the leaves of each tree

    # Get number of leaves and ids of the leaves on each tree
    for tree in 1:n_trees
        leaves[tree] = findall(node -> evo_model.trees[tree].split[node] == false && (node == 1 || evo_model.trees[tree].split[floor(Int, node / 2)] == true), 1:length(evo_model.trees[tree].split))
        n_leaves[tree] = length(leaves[tree])
    end

    splits = Matrix{Any}(undef, n_trees, length(evo_model.trees[2].split)) # storing the feature number and splitpoint index for each split node
    splits_ordered = Array{Vector}(undef, n_feats) # splitpoints for each feature

    n_splits = zeros(Int64, n_feats)
    [splits_ordered[feat] = [] for feat in 1:n_feats]

    for tree in 1:n_trees
        for node in eachindex(evo_model.trees[tree].split)
            if evo_model.trees[tree].split[node] == true
                splits[tree, node] = [evo_model.trees[tree].feat[node], evo_model.trees[tree].cond_float[node]] # save feature and split value
                push!(splits_ordered[evo_model.trees[tree].feat[node]], evo_model.trees[tree].cond_float[node]) # push split value to splits_ordered
            end
        end
    end
    [unique!(sort!(splits_ordered[feat])) for feat in 1:n_feats] # sort splits_ordered and remove copies
    [n_splits[feat] = length(splits_ordered[feat]) for feat in 1:n_feats] # store number of split points

    for tree in 1:n_trees
        for node in eachindex(evo_model.trees[tree].split)
            if evo_model.trees[tree].split[node] == true
                
                feature::Int = splits[tree, node][1]
                value = splits[tree, node][2]

                splits[tree, node][2] = searchsortedfirst(splits_ordered[feature], value)

            end
        end
    end


    b = Array{Array}(undef, n_trees)
    [b[tree] = evo_model.trees[tree].pred for tree in 1:n_trees]
    # -------------------------------------------------------
    # in the Linear decision tree, there should be gradient in addition to coefficient
    a = Array{Array}(undef, n_trees)
    [a[tree] = zeros(length(evo_model.trees[tree].pred), n_feats) for tree in 1:n_trees]
    # -------------------------------------------------------
    
    split_nodes = Array{Array}(undef, n_trees)
    [split_nodes[tree] = evo_model.trees[tree].split for tree in 1:n_trees]

    return LTEModel(n_trees, n_feats, n_leaves, leaves, splits, splits_ordered, n_splits, a, b, split_nodes, Array{Array}(undef, n_trees))

end

extract_evotrees_info (generic function with 1 method)

In [457]:
LTE = extract_evotrees_info(evo_model; tree_limit=length(evo_model.trees))

LTEModel(201, 5, [1, 15, 16, 16, 15, 16, 15, 14, 16, 16  …  16, 14, 15, 15, 16, 16, 12, 15, 12, 16], Array[[1], [9, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [11, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31], [8, 10, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]  …  [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 28, 29, 30, 31], [13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31], [13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],

No changes in the function `init_TEModel!()`, the only change is that we work with `LTEModel` now.

In [458]:
function children(id::Int, leaf_dict::Dict, max::Int)

    result::Vector{Int} = []

    function inner(num)
        if num <= max
            leaf_index = get(leaf_dict, num, 0)
            if leaf_index != 0
                push!(result, leaf_index)
            end
            inner(num << 1)
            inner(num << 1 + 1)
        end
    end

    inner(id)

    return result

end

children (generic function with 1 method)

In [459]:
function init_LTEModel!(LTE::LTEModel)

    leaf_dict = Array{Dict}(undef, LTE.n_trees)
    [leaf_dict[tree] = Dict([(LTE.leaves[tree][leaf], leaf) for leaf in eachindex(LTE.leaves[tree])]) for tree in 1:LTE.n_trees]

    # pre-compute all children for all active nodes of all trees
    for tree in 1:LTE.n_trees
        
        nodes_with_split = findall(split -> split == true, LTE.split_nodes[tree])
        LTE.child_leaves[tree] = Array{Any}(undef, maximum(LTE.leaves[tree]))

        for node in [nodes_with_split; LTE.leaves[tree]]
            LTE.child_leaves[tree][node] = children(node, leaf_dict[tree], last(LTE.leaves[tree]))
        end
    end
end


init_LTEModel! (generic function with 1 method)

In [460]:
L_bounds = -0.5*ones(LTE.n_feats);
U_bounds = 0.5*ones(LTE.n_feats);
d_bounds = [-1, 1];

In [461]:
function calculate_bounds(model::JuMP.Model, LTE::LTEModel, leaf, tree, d_bounds)

    # Something is wrong there 
    # ------------------------------------

    @objective(model, Max,  LTE.a[tree][LTE.leaves[tree][leaf], :]'*model[:x]+LTE.b[tree][LTE.leaves[tree][leaf]] - model[:t][tree])

    optimize!(model)
    upper_bound = objective_value(model)

    set_objective_sense(model, MIN_SENSE)
    optimize!(model)

    lower_bound = objective_value(model)
    
    return [upper_bound, lower_bound]
    
end

calculate_bounds (generic function with 1 method)

In [462]:
function LTE_formulate!(model::JuMP.Model, LTE::LTEModel, U_bounds, L_bounds; bound_tightening = "quadratic", d_bounds=nothing)

    empty!(model);
    if bound_tightening == "output" @assert !isnothing(d_bounds) "Bounds for forest output must be provided." end

    init_LTEModel!(LTE);
    

    # (2.2.8)
    @variable(model, x[1:LTE.n_feats]);
    @constraint(model, [i = 1:LTE.n_feats], x[i] <= U_bounds[i]);
    @constraint(model, [i = 1:LTE.n_feats], x[i] >= L_bounds[i]);

    # (2.2.9)
    @variable(model, y[i = 1:LTE.n_feats, 1:LTE.n_splits[i]], Bin);

    # (2.2.10)
    @variable(model, z[tree = 1:LTE.n_trees, 1:LTE.n_leaves[tree]], Bin); 

    @variable(model, d);

    # (2.2.5)
    @constraint(model, [i = 1:LTE.n_feats, j = 1:(LTE.n_splits[i]-1)], y[i,j] <= y[i, j+1]); 

    v = []
    for i = 1:length(LTE.splits_ordered)
        push!(v, vcat(L_bounds[i], LTE.splits_ordered[i], U_bounds[i]));
    end

    # (2.2.6)
    @constraint(model, [i = 1:LTE.n_feats], x[i] >=  v[i][1] + sum((v[i][j] - v[i][j-1])*(1 - y[i, j-1]) for j = 2 : LTE.n_splits[i]+1));

    # (2.2.7)
    @constraint(model, [i = 1:LTE.n_feats], x[i] <=  v[i][LTE.n_splits[i]+2] + sum((v[i][j] - v[i][j+1])* y[i, j-1] for j = 2 : LTE.n_splits[i]+1));

    # (2.2.2)
    @constraint(model, [tree = 1:LTE.n_trees], sum(z[tree, i] for i=1:LTE.n_leaves[tree]) == 1);
    
    for tree in 1:LTE.n_trees
        for current_node in findall(s -> s==true, LTE.split_nodes[tree])

            right_leaves = LTE.child_leaves[tree][current_node << 1 + 1];
            left_leaves = LTE.child_leaves[tree][current_node << 1];

            current_feat, current_splitpoint_index = LTE.splits[tree, current_node];

            # (2.2.3)
            @constraint(model, sum(model[:z][tree, leaf] for leaf in right_leaves) <= 1 - model[:y][current_feat, current_splitpoint_index]);

            # (2.2.4)
            @constraint(model, sum(model[:z][tree, leaf] for leaf in left_leaves) <= model[:y][current_feat, current_splitpoint_index]);
        end
    end

    if bound_tightening=="quadratic"
        # (2.2.1)
        @constraint(model, d == sum((LTE.a[tree][LTE.leaves[tree][leaf], :]'*x+LTE.b[tree][LTE.leaves[tree][leaf]])*z[tree, leaf] for tree = 1:LTE.n_trees, leaf = 1:LTE.n_leaves[tree]));
    
    elseif  bound_tightening=="output"

        # It doesn't work
        # ---------------------
        @variable(model, t[1:LTE.n_trees]);        
        @constraint(model, [tree = 1:LTE.n_trees], d_bounds[1]<=t[tree]<=d_bounds[2]);
        bounds = [[calculate_bounds(model, LTE, leaf, tree, d_bounds) for leaf in 1:LTE.n_leaves[tree]] for tree in 1:LTE.n_trees];
        # (2.2.12)
        @constraint(model, [tree = 1:LTE.n_trees, leaf = 1:LTE.n_leaves[tree]], LTE.a[tree][LTE.leaves[tree][leaf], :]'*x+LTE.b[tree][LTE.leaves[tree][leaf]] - t[tree] <=bounds[tree][leaf][1]*(1 - z[tree, leaf]));
        
        # (2.2.13)
        @constraint(model, [tree = 1:LTE.n_trees, leaf = 1:LTE.n_leaves[tree]], LTE.a[tree][LTE.leaves[tree][leaf], :]'*x+LTE.b[tree][LTE.leaves[tree][leaf]] - t[tree] >=-bounds[tree][leaf][2]*(1 - z[tree, leaf]));
        
        @constraint(model,  sum(t[tree] for tree = 1:LTE.n_trees) == d);
        
        # ---------------------
    end      
   end

LTE_formulate! (generic function with 2 methods)

In [463]:
model = Model(() -> Gurobi.Optimizer());

LTE_formulate!(model, LTE, U_bounds, L_bounds, bound_tightening = "quadratic")

0.002617006655782461 z[1,1] + 0.03208799660205841 z[2,1] + 0.025356872007250786 z[2,2] + 0.03195912018418312 z[2,3] + 0.022056153044104576 z[2,4] + 0.010642416775226593 z[2,5] + 0.007492331322282553 z[2,6] - 0.0033112093806266785 z[2,7] + 0.021318046376109123 z[2,8] + 0.005893046502023935 z[2,9] + 0.00015210488345474005 z[2,10] - 0.008136735297739506 z[2,11] - 0.0033859473187476397 z[2,12] - 0.011718281544744968 z[2,13] - 0.006805223412811756 z[2,14] - 0.015342223457992077 z[2,15] + 0.016502728685736656 z[3,1] + 0.00887283869087696 z[3,2] - 0.0015014056116342545 z[3,3] - 0.015306669287383556 z[3,4] + 0.016169315204024315 z[3,5] + 0.004910515155643225 z[3,6] + 0.0004831280966755003 z[3,7] - 0.007588396314531565 z[3,8] + 0.008103816770017147 z[3,9] + 0.0005962153081782162 z[3,10] - 0.001233614981174469 z[3,11] - 0.009158176369965076 z[3,12] - 0.004883254878222942 z[3,13] - 0.014234708622097969 z[3,14] - [[...2910 terms omitted...]] - 2.768239482975332e-6 z[199,15] - 0.0003050238010473549

In [464]:
@objective(model, Min, model[:d])

d

In [465]:
model

A JuMP Model
Minimization problem with:
Variables: 3286
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 201 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 10 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 5852 constraints
`QuadExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 3280 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: d, x, y, z

In [466]:
optimize!(model)

In [467]:
objective_value(model)

-0.4231592458540945

In [468]:
EvoTrees.predict(evo_model, value.(model[:x])')[1] 

-0.4231592f0